### Improving the accuracy of the MNISTmodel.

In [1]:
import tensorflow as tf

In [2]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [76]:
# Parameters
learning_rate = 1e-10
training_epochs = 5
batch_size = 100
display_step = 1
input_size = 784
output_size = 10

In [4]:
# tf Graph Input
x = tf.placeholder(tf.float32, [None, input_size]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, output_size]) # 0-9 digits recognition => 10 classes

# Set model weights
#W = tf.Variable(tf.random_normal([input_size,output_size],mean=0.0,stddev=1.0, dtype=tf.float32))
#b = tf.Variable(tf.random_normal([output_size],mean=0.0,stddev=1.0, dtype=tf.float32))

In [63]:
dimensions = [512]
current_input = x
n_input = input_size

# We're going to keep every matrix we create so let's create a list to hold them all
Ws = []

# We'll create a for loop to create each layer:
for layer_i, n_output in enumerate(dimensions):

    # just like in the last session,
    # we'll use a variable scope to help encapsulate our variables
    # This will simply prefix all the variables made in this scope
    # with the name we give it.
    with tf.variable_scope("FC/layer/{}".format(layer_i),reuse=True):

        # Create a weight matrix which will increasingly reduce
        # down the amount of information in the input by performing
        # a matrix multiplication
        W = tf.get_variable(
            name='W',
            shape=[n_input, n_output],
            initializer=tf.random_normal_initializer(mean=0.0, stddev=0.02))
        
        B = tf.get_variable(
            name='B',
            shape=[n_output],
            initializer=tf.random_normal_initializer(mean=0.0, stddev=0.02))
        

        # Now we'll multiply our input by our newly created W matrix
        # and add the bias
        H = tf.nn.bias_add(
            name='H',
            value=tf.matmul(current_input, W),
            bias=B)

        # And then use a relu activation function on its output
        H  = tf.nn.relu(H)
        
        #add a dropout layer
        current_input = tf.nn.dropout(H,0.8)

        # Finally we'll store the weight matrix so we can build the decoder.
        Ws.append(W)

        # We'll also replace n_input with the current n_output, so that on the
        # next iteration, our new number inputs will be correct.
        n_input = n_output
        
        print(current_input.get_shape().as_list())

[None, 512]


In [69]:
## we have our final layer
with tf.variable_scope("FC", reuse=True):
    Wfinal = tf.get_variable(
            name='Wfinal',
            shape=[dimensions[-1], output_size],
            initializer=tf.random_normal_initializer(mean=0.0, stddev=0.02))

    Bfinal = tf.get_variable(
            name='Bfinal',
            shape=[output_size],
            initializer=tf.random_normal_initializer(mean=0.0, stddev=0.02))

    Hfinal = tf.nn.bias_add(
            name='Hfinal',
            value=tf.matmul(current_input, Wfinal),
            bias=Bfinal)
    
    #Hfinal = tf.nn.dropout(Hfinal,0.8)
 
    pred = tf.nn.softmax(Hfinal) # Softmax 

print(pred.get_shape().as_list())

[None, 10]


In [70]:
with tf.variable_scope("FC", reuse=True):
    #mean square error
    cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred + 1e-12)))

    #adamoptimizer
    optimizer =  tf.train.AdamOptimizer(learning_rate).minimize(cost) 

In [71]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [72]:
# no of correct prediction
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [77]:
sess = tf.Session()
# Run the initializer
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop) and cost op (to get loss value)
        _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
        # Compute average loss
        avg_cost += c / total_batch
    # Display logs per epoch step
    if (epoch+1) % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1), "cost= ",(avg_cost))
        acc = sess.run(accuracy,feed_dict = {x: batch_xs, y: batch_ys})
        print ("accuracy = ",acc)
        ##print("Accuracy:", acc.eval(sesion = sess))

print("Optimization Finished!")

Epoch: 0001 cost=  25.423209375
accuracy =  0.93
Epoch: 0002 cost=  16.7542151945
accuracy =  0.99
Epoch: 0003 cost=  14.8089696351
accuracy =  0.98
Epoch: 0004 cost=  13.2353006083
accuracy =  0.99
Epoch: 0005 cost=  13.0790489027
accuracy =  0.98
Optimization Finished!


In [78]:
##run it on test Set
prd = sess.run(pred,feed_dict = {x: mnist.test.images, y: mnist.test.labels})
acc1 = sess.run(accuracy,feed_dict = {x: mnist.test.images, y: mnist.test.labels})
print ("accuracy on test set: ",acc1)

accuracy on test set:  0.9596
